<a href="https://colab.research.google.com/github/AhadAli451/PIAIC-Quarter-2-Projects/blob/main/03_Function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
!pip install -U -q "google-generativeai>=0.7.2"

In [40]:
from google.colab import userdata
import google.generativeai as genai

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

In [41]:
def enable_fan():
    """Turn on the fan system."""
    print("FANBOT: Fan enabled.")


def set_fan_speed(speed_level: int):
    """Set the fan speed. Fan must be enabled for this to work.

    Args:
        speed_level (int): The speed level to set, typically between 1 and 5.
    """
    print(f"FANBOT: Fan speed set to level {speed_level}.")


def stop_fan():
    """Turn off the fan."""
    print("FANBOT: Fan turned off.")


fan_controls = [enable_fan, set_fan_speed, stop_fan]
instruction = "You are a helpful fan system bot. You can turn the fan on and off, and you can set the speed. Do not perform any other tasks."

model = genai.GenerativeModel(
    "models/gemini-1.5-pro",
    tools=fan_controls,
    system_instruction=instruction
)

chat = model.start_chat()

In [42]:
from google.generativeai.types import content_types
from collections.abc import Iterable


def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

In [43]:
tool_config = tool_config_from_mode("none")

response = chat.send_message(
    "Hello fan-bot, what can you do?", tool_config=tool_config
)
print(response.text)

I can turn the fan on and off, and I can set the fan speed.



In [45]:
tool_config = tool_config_from_mode("auto")

response = chat.send_message("on the fan!", tool_config=tool_config)
# print(response.parts[0])
# chat.rewind();  # You are not actually calling the function, so remove this from the history.

In [46]:
print(response.parts[0])
chat.rewind();  # You are not actually calling the function, so remove this from the history.

function_call {
  name: "enable_fan"
  args {
  }
}



In [47]:
available_fns = ["set_fan_speed", "stop_fan"]

tool_config = tool_config_from_mode("any", available_fns)

response = chat.send_message("set the speed of fan", tool_config=tool_config)
print(response.parts[0])

function_call {
  name: "set_fan_speed"
  args {
    fields {
      key: "speed_level"
      value {
        number_value: 3
      }
    }
  }
}



In [48]:
# available_fns = ["enable_lights"]
available_fns = ["enable_fan","set_fan_speed", "stop_fan"]
tool_config = tool_config_from_mode("any", available_fns)

auto_chat = model.start_chat(enable_automatic_function_calling=True)
response = auto_chat.send_message("the room temperature is very high set the speed of fan and then stop fan", tool_config=tool_config)

FANBOT: Fan enabled.
FANBOT: Fan speed set to level 3.0.
FANBOT: Fan turned off.
